# PP2A - Single gene analysis in GP2 data

## Description

Using "individual level data"

### 0. Getting Started

- Loading Python libraries
- Defining functions
- Installing packages

### 1. Copy data from workspace to cloud environment

### 2. Extract PP2A

### 3. Annotate PP2A variants

### 4. Extract coding/non-syn variants

### 5. Calculate frequency in cases versus controls

### 6. Calculate frequency (homozygotes) in cases versus controls

### 7. Save out results

## Getting Started

### Loading Python libraries

In [1]:
# Use the os package to interact with the environment
import os

# Bring in Pandas for Dataframe functionality
import pandas as pd

# Numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

#Import Sys
import sys as sys

### Defining functions

In [2]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
    
def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for printing a query before executing it
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

### Set paths

In [3]:
# Set up billing project and data path variables
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

## Print the information to check we are in the proper release and billing 
## This will be different for you, the user, depending on the billing project your workspace is on
print('Billing and Workspace')
print(f'Workspace Name: {WORKSPACE_NAME}')
print(f'Billing Project: {BILLING_PROJECT_ID}')
print(f'Workspace Bucket, where you can upload and download data: {WORKSPACE_BUCKET}')
print('')


## GP2 v3.0
## Explicitly define release v3.0 path 
GP2_RELEASE_PATH = 'gs://gp2tier2/release3_31102022'
GP2_CLINICAL_RELEASE_PATH = f'{GP2_RELEASE_PATH}/clinical_data'
GP2_RAW_GENO_PATH = f'{GP2_RELEASE_PATH}/raw_genotypes'
GP2_IMPUTED_GENO_PATH = f'{GP2_RELEASE_PATH}/imputed_genotypes'
print('GP2 v3.0')
print(f'Path to GP2 v3.0 Clinical Data: {GP2_CLINICAL_RELEASE_PATH}')
print(f'Path to GP2 v3.0 Raw Genotype Data: {GP2_RAW_GENO_PATH}')
print(f'Path to GP2 v3.0 Imputed Genotype Data: {GP2_IMPUTED_GENO_PATH}')


## AMP-PD v3.0
## Explicitly define release v3.0 path 
AMP_RELEASE_PATH = 'gs://amp-pd-data/releases/2022_v3release_1115'
AMP_CLINICAL_RELEASE_PATH = f'{AMP_RELEASE_PATH}/clinical'

AMP_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS'
AMP_WGS_RELEASE_PLINK_PATH = os.path.join(AMP_WGS_RELEASE_PATH, 'plink')
AMP_WGS_RELEASE_PLINK_PFILES = os.path.join(AMP_WGS_RELEASE_PLINK_PATH, 'pfiles')

print('AMP-PD v3.0')
print(f'Path to AMP-PD v3.0 Clinical Data: {AMP_CLINICAL_RELEASE_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PFILES}')
print('')

Billing and Workspace
Workspace Name: PP2A
Billing Project: terra-02e7cd23
Workspace Bucket, where you can upload and download data: gs://fc-a2382a6f-59cf-4cf3-b70e-8cd83806d5e0

GP2 v3.0
Path to GP2 v3.0 Clinical Data: gs://gp2tier2/release3_31102022/clinical_data
Path to GP2 v3.0 Raw Genotype Data: gs://gp2tier2/release3_31102022/raw_genotypes
Path to GP2 v3.0 Imputed Genotype Data: gs://gp2tier2/release3_31102022/imputed_genotypes
AMP-PD v3.0
Path to AMP-PD v3.0 Clinical Data: gs://amp-pd-data/releases/2022_v3release_1115/clinical
Path to AMP-PD v3.0 WGS Data: gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS/plink
Path to AMP-PD v3.0 WGS Data: gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS/plink/pfiles



### Install packages

#### Install Plink 1.9 and Plink 2.0

In [4]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink; then
echo "Plink1.9 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink \n    -------"
wget -N http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 
unzip -o plink_linux_x86_64_20190304.zip
echo -e "\n plink downloaded and unzipped in /home/jupyter/tools \n "

fi


if test -e /home/jupyter/tools/plink2; then
echo "Plink2 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink2 \n    -------"
wget -N https://s3.amazonaws.com/plink2-assets/alpha3/plink2_linux_avx2_20220603.zip
unzip -o plink2_linux_avx2_20220603.zip
echo -e "\n plink2 downloaded and unzipped in /home/jupyter/tools \n "

fi

Plink1.9 is already installed in /home/jupyter/tools/
Plink2 is already installed in /home/jupyter/tools/


In [5]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Remote restrictions

In [6]:
%%bash

# chmod plink 1.9 
chmod u+x /home/jupyter/tools/plink

In [7]:
%%bash

# chmod plink 2.0
chmod u+x /home/jupyter/tools/plink2

### Install ANNOVAR

In [8]:
%%bash

# Install ANNOVAR:
# https://www.openbioinformatics.org/annovar/annovar_download_form.php

if test -e /home/jupyter/tools/annovar; then

echo "annovar is already installed in /home/jupyter/tools/"
else
echo "annovar is not installed"
cd /home/jupyter/tools/

wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

tar xvfz annovar.latest.tar.gz

fi

annovar is already installed in /home/jupyter/tools/


In [9]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Install ANNOVAR: Download sources of annotation 

In [10]:
%%bash

cd /home/jupyter/tools/annovar/

perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20140902 humandb/
#perl annotate_variation.pl -buildver hg38 -downdb cytoBand humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ensGene humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar exac03 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar avsnp147 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp30a humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar gnomad211_genome humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ljb26_all humandb/

NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGene.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneMrna.fa.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneVersion.txt.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished downloading annotation files for hg38 build version, with files saved at the 'humandb' directory
NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.idx.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished d

In [11]:
%%bash
ls /home/jupyter/tools/annovar/

annotate_variation.pl
coding_change.pl
convert2annovar.pl
example
humandb
retrieve_seq_from_fasta.pl
table_annovar.pl
variants_reduction.pl


## Copy data from GP2 bucket to workspace

In [12]:
# Make a directory
print("Making a working directory")
WORK_DIR = f'/home/jupyter/PP2A_GP2/'
shell_do(f'mkdir -p {WORK_DIR}') # f' significa f-string - contiene expresiones para ejecutar el codigo

Making a working directory


Executing: mkdir -p /home/jupyter/PP2A_GP2/


In [13]:
# Check directory where AMP-PD data is
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_IMPUTED_GENO_PATH}')

Executing: gsutil -u terra-02e7cd23 ls gs://gp2tier2/release3_31102022/imputed_genotypes


gs://gp2tier2/release3_31102022/imputed_genotypes/AAC/
gs://gp2tier2/release3_31102022/imputed_genotypes/AFR/
gs://gp2tier2/release3_31102022/imputed_genotypes/AJ/
gs://gp2tier2/release3_31102022/imputed_genotypes/AMR/
gs://gp2tier2/release3_31102022/imputed_genotypes/CAS/
gs://gp2tier2/release3_31102022/imputed_genotypes/EAS/
gs://gp2tier2/release3_31102022/imputed_genotypes/EUR/
gs://gp2tier2/release3_31102022/imputed_genotypes/MDE/
gs://gp2tier2/release3_31102022/imputed_genotypes/SAS/


In [14]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_IMPUTED_GENO_PATH}/EUR/chr9* {WORK_DIR}')

Executing: gsutil -u terra-02e7cd23 -m cp -r gs://gp2tier2/release3_31102022/imputed_genotypes/EUR/chr9* /home/jupyter/PP2A_GP2/


Copying gs://gp2tier2/release3_31102022/imputed_genotypes/EUR/chr9_EUR_release3.psam...
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/EUR/chr9_EUR_release3.log...
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/EUR/chr9_EUR_release3.pgen...
Copying gs://gp2tier2/release3_31102022/imputed_genotypes/EUR/chr9_EUR_release3.pvar...
\ [4/4 files][  4.4 GiB/  4.4 GiB] 100% Done  38.5 MiB/s ETA 00:00:00           
Operation completed over 4 objects/4.4 GiB.                                      


### Create a covariate file with GP2 data

In [15]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_CLINICAL_RELEASE_PATH}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_CLINICAL_RELEASE_PATH}/master_key_release3_final.csv {WORK_DIR}')

Executing: gsutil -u terra-02e7cd23 ls gs://gp2tier2/release3_31102022/clinical_data


gs://gp2tier2/release3_31102022/clinical_data/
gs://gp2tier2/release3_31102022/clinical_data/master_key_release3_final.csv
gs://gp2tier2/release3_31102022/clinical_data/release3_31102022_data_dictionary.csv


Executing: gsutil -u terra-02e7cd23 -m cp -r gs://gp2tier2/release3_31102022/clinical_data/master_key_release3_final.csv /home/jupyter/PP2A_GP2/


Copying gs://gp2tier2/release3_31102022/clinical_data/master_key_release3_final.csv...
/ [1/1 files][  1.6 MiB/  1.6 MiB] 100% Done                                    
Operation completed over 1 objects/1.6 MiB.                                      


In [16]:
cov = pd.read_csv(f'{WORK_DIR}/master_key_release3_final.csv')
cov.columns

Index(['GP2ID', 'GP2sampleID', 'manifest_id', 'phenotype', 'pheno_for_qc',
       'other_pheno', 'sex_for_qc', 'age', 'age_of_onset', 'age_at_diagnosis',
       'age_at_death', 'race_for_qc', 'family_history_for_qc', 'region_for_qc',
       'study', 'pruned', 'pruned_reason', 'label', 'related'],
      dtype='object')

In [17]:
cov_reduced = cov[['GP2sampleID','GP2sampleID','sex_for_qc', 'age', 'phenotype']]
cov_reduced.columns = ['FID','IID', 'SEX','AGE','PHENO']
cov_reduced

,FID,IID,SEX,AGE,PHENO
0,SYNAPS-KZ_000001_s1,SYNAPS-KZ_000001_s1,2,64.0,Control
1,SYNAPS-KZ_000002_s1,SYNAPS-KZ_000002_s1,1,53.0,Control
2,SYNAPS-KZ_000003_s1,SYNAPS-KZ_000003_s1,2,59.0,Control
3,SYNAPS-KZ_000004_s1,SYNAPS-KZ_000004_s1,1,57.0,Control
4,SYNAPS-KZ_000005_s1,SYNAPS-KZ_000005_s1,1,59.0,Control
...,...,...,...,...,...
16852,PPMI_000789_s1,PPMI_000789_s1,2,NaN,Other
16853,PPMI_000781_s1,PPMI_000781_s1,1,57.0,Other
16854,PPMI_000818_s1,PPMI_000818_s1,1,60.0,Other
16855,PPMI_000841_s1,PPMI_000841_s1,2,68.0,Other


In [18]:
conditions = [(cov_reduced['PHENO'] == "Case"),
    (cov_reduced['PHENO'] == "Control")]

In [19]:
choices = [2,1]
cov_reduced['PHENOTYPE'] = np.select(conditions, choices, default=-9).astype(np.int64)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


##### cov_reduced.reset_index(inplace=True)
cov_reduced.drop(columns=["index"], inplace=True)

In [20]:
cov_reduced.drop(columns=['PHENO'], inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [21]:
sex = cov_reduced[['FID','IID','SEX']]
sex.to_csv(f'{WORK_DIR}/SEX.txt',index=False, sep="\t")

In [22]:
pheno = cov_reduced[['FID','IID','PHENOTYPE']]
pheno.to_csv(f'{WORK_DIR}/PHENO.txt',index=False, sep="\t")

## Extract PP2A

In [23]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

# PODXL: gene positions on hg38 (from https://useast.ensembl.org/index.html)
/home/jupyter/tools/plink2 \
--pfile chr9_EUR_release3 \
--chr 9 \
--from-bp 129110950  \
--to-bp 129148946 \
--make-bed \
--out pheno_PP2A_EUR

PLINK v2.00a3.3LM AVX2 Intel (3 Jun 2022)      www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pheno_PP2A_EUR.log.
Options in effect:
  --chr 9
  --from-bp 129110950
  --make-bed
  --out pheno_PP2A_EUR
  --pfile chr9_EUR_release3
  --to-bp 129148946

Start time: Thu Jun  1 14:03:35 2023
14998 MiB RAM detected; reserving 7499 MiB for main workspace.
Using up to 4 compute threads.
11002 samples (4271 females, 6731 males; 11002 founders) loaded from
chr9_EUR_release3.psam.
859295 variants loaded from chr9_EUR_release3.pvar.
1 binary phenotype loaded (6852 cases, 3875 controls).
328 variants remaining after main filters.
Writing pheno_PP2A_EUR.fam ... done.
Writing pheno_PP2A_EUR.bim ... done.
Writing pheno_PP2A_EUR.bed ... 0%done.
End time: Thu Jun  1 14:03:36 2023


### Visualize plink files bim, fam and bed

In [24]:
%%bash

# Visualize bim file
WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head pheno_PP2A_EUR.bim

9	chr9:129110955:G:C	0	129110955	C	G
9	chr9:129110977:A:C	0	129110977	C	A
9	chr9:129111059:G:A	0	129111059	A	G
9	chr9:129111157:A:T	0	129111157	T	A
9	chr9:129111178:T:C	0	129111178	C	T
9	chr9:129111184:G:C	0	129111184	C	G
9	chr9:129111323:C:G	0	129111323	G	C
9	chr9:129111355:C:G	0	129111355	G	C
9	chr9:129111392:G:C	0	129111392	C	G
9	chr9:129111663:T:C	0	129111663	C	T


In [25]:
%%bash

# Visualize fam file
WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head pheno_PP2A_EUR.fam

0	APGS_000001_s1	0	0	1	2
0	APGS_000002_s1	0	0	1	2
0	APGS_000003_s1	0	0	1	2
0	APGS_000005_s1	0	0	1	2
0	APGS_000006_s1	0	0	1	2
0	APGS_000007_s1	0	0	1	2
0	APGS_000009_s1	0	0	1	2
0	APGS_000010_s1	0	0	1	2
0	APGS_000012_s1	0	0	2	2
0	APGS_000013_s1	0	0	1	2


In [26]:
%%bash

# Visualize fam file
WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head pheno_PP2A.bed

l��������������������������߻�����������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������/��������ߺ����{�����������������������������������;���������������������������������������������/��������ߺ����{�����������������������������������;�������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������/��������ߺ����{�����������������������������������;�������������������������������������������������������������������������������������������������������         �                                     �    @            @             @      �������������������������������������������/��������ߺ����{�����������������������������������;��������

In [27]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

# Turn binary files into VCF
/home/jupyter/tools/plink \
--bfile pheno_PP2A_EUR \
--recode vcf-fid \
--out pheno_PP2A_EUR

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pheno_PP2A_EUR.log.
Options in effect:
  --bfile pheno_PP2A_EUR
  --out pheno_PP2A_EUR
  --recode vcf-fid

14998 MB RAM detected; reserving 7499 MB for main workspace.
328 variants loaded from .bim file.
11002 people (6731 males, 4271 females) loaded from .fam.
10727 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 11002 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45%46%47%48%49%50%51%52%53%54%55%56%57%58%59%60%61%62%63%64%

## Annotate PP2A variants

In [28]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR
export PATH=$PATH:/home/jupyter/tools/rvtests/third/tabix-0.2.6/

### Bgzip and Tabix
bgzip pheno_PP2A.vcf

tabix -f -p vcf pheno_PP2A.vcf.gz
tabix -f -p vcf pheno_PP2A.vcf.gz

[bgzip] can't create pheno_PP2A.vcf.gz: File exists


In [29]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

### Annotate variants using ANNOVAR: https://annovar.openbioinformatics.org/en/latest/ 
perl /home/jupyter/tools/annovar/table_annovar.pl pheno_PP2A.vcf.gz /home/jupyter/tools/annovar/humandb/ -buildver hg38 \
-out pheno_PP2A.annovar \
-remove -protocol refGene,clinvar_20140902 \
-operation g,f \
--nopolish \
-nastring . \
-vcfinput


NOTICE: Running with system command <convert2annovar.pl  -includeinfo -allsample -withfreq -format vcf4 pheno_PP2A.vcf.gz > pheno_PP2A.annovar.avinput>
NOTICE: Finished reading 335 lines from VCF file
NOTICE: A total of 328 locus in VCF file passed QC threshold, representing 303 SNPs (226 transitions and 77 transversions) and 25 indels/substitutions
NOTICE: Finished writing allele frequencies based on 3333606 SNP genotypes (2486452 transitions and 847154 transversions) and 275050 indels/substitutions for 11002 samples

NOTICE: Running with system command </home/jupyter/tools/annovar/table_annovar.pl pheno_PP2A.annovar.avinput /home/jupyter/tools/annovar/humandb/ -buildver hg38 -outfile pheno_PP2A.annovar -remove -protocol refGene,clinvar_20140902 -operation g,f --nopolish -nastring . -otherinfo>
-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=refGene

NOTICE: Running with system command <annotate_variation.pl -geneanno -buildve

## Extract coding and non-syn variants

In [30]:
# Visualize multianno file
PP2A = pd.read_csv(f'{WORK_DIR}/pheno_PP2A.annovar.hg38_multianno.txt', sep = '\t')
PP2A.head()

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo11005,Otherinfo11006,Otherinfo11007,Otherinfo11008,Otherinfo11009,Otherinfo11010,Otherinfo11011,Otherinfo11012,Otherinfo11013,Otherinfo11014
0,9,129110955,129110955,G,C,UTR5,PTPA,NM_021131:c.-646G>C,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
1,9,129110977,129110977,A,C,UTR5,PTPA,NM_021131:c.-624A>C,.,.,...,1/1,0/0,0/0,0/0,0/1,0/0,0/0,0/0,0/0,0/1
2,9,129111059,129111059,G,A,intronic,PTPA,.,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
3,9,129111157,129111157,A,T,intronic,PTPA,.,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
4,9,129111178,129111178,T,C,intronic,PTPA,.,.,.,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [31]:
# Filter exonic variants
coding = PP2A[PP2A['Func.refGene'] == 'exonic']
coding.count()

Chr               4
Start             4
End               4
Ref               4
Alt               4
                 ..
Otherinfo11010    4
Otherinfo11011    4
Otherinfo11012    4
Otherinfo11013    4
Otherinfo11014    4
Length: 11025, dtype: int64

In [32]:
coding.head(10)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo11005,Otherinfo11006,Otherinfo11007,Otherinfo11008,Otherinfo11009,Otherinfo11010,Otherinfo11011,Otherinfo11012,Otherinfo11013,Otherinfo11014
139,9,129127987,129127987,A,G,exonic,PTPA,.,synonymous SNV,PTPA:NM_178001:exon4:c.A240G:p.E80E,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
140,9,129128008,129128008,G,A,exonic,PTPA,.,synonymous SNV,PTPA:NM_178001:exon4:c.G261A:p.Q87Q,...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
260,9,129142498,129142498,C,T,exonic,PTPA,.,synonymous SNV,"PTPA:NM_001271832:exon8:c.C753T:p.A251A,PTPA:N...",...,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0
312,9,129147457,129147457,C,T,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon9:c.C878T:p.S293L,PTPA:N...",...,1/1,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [33]:
coding = PP2A[PP2A['Func.refGene'] == 'intronic']
coding.count()

Chr               304
Start             304
End               304
Ref               304
Alt               304
                 ... 
Otherinfo11010    304
Otherinfo11011    304
Otherinfo11012    304
Otherinfo11013    304
Otherinfo11014    304
Length: 11025, dtype: int64

In [34]:
coding = PP2A[PP2A['Func.refGene'] == 'UTR5']
coding.count()

Chr               5
Start             5
End               5
Ref               5
Alt               5
                 ..
Otherinfo11010    5
Otherinfo11011    5
Otherinfo11012    5
Otherinfo11013    5
Otherinfo11014    5
Length: 11025, dtype: int64

In [35]:
coding = PP2A[PP2A['Func.refGene'] == 'UTR3']
coding.count()

Chr               15
Start             15
End               15
Ref               15
Alt               15
                  ..
Otherinfo11010    15
Otherinfo11011    15
Otherinfo11012    15
Otherinfo11013    15
Otherinfo11014    15
Length: 11025, dtype: int64

In [36]:
# Filter exonic and non-syn vars
coding_nonsynonymous = PP2A[(PP2A['Func.refGene'] == 'exonic') & (PP2A['ExonicFunc.refGene'] == 'nonsynonymous SNV')]
coding_nonsynonymous.count()

Chr               1
Start             1
End               1
Ref               1
Alt               1
                 ..
Otherinfo11010    1
Otherinfo11011    1
Otherinfo11012    1
Otherinfo11013    1
Otherinfo11014    1
Length: 11025, dtype: int64

In [37]:
coding_nonsynonymous.head(1)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo11005,Otherinfo11006,Otherinfo11007,Otherinfo11008,Otherinfo11009,Otherinfo11010,Otherinfo11011,Otherinfo11012,Otherinfo11013,Otherinfo11014
312,9,129147457,129147457,C,T,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon9:c.C878T:p.S293L,PTPA:N...",...,1/1,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


In [38]:
print(coding_nonsynonymous)

     Chr      Start        End Ref Alt Func.refGene Gene.refGene  \
312    9  129147457  129147457   C   T       exonic         PTPA   

    GeneDetail.refGene ExonicFunc.refGene  \
312                  .  nonsynonymous SNV   

                                      AAChange.refGene  ... Otherinfo11005  \
312  PTPA:NM_001271832:exon9:c.C878T:p.S293L,PTPA:N...  ...            1/1   

     Otherinfo11006 Otherinfo11007 Otherinfo11008  Otherinfo11009  \
312             0/0            0/0            0/0             0/0   

     Otherinfo11010 Otherinfo11011 Otherinfo11012 Otherinfo11013  \
312             0/0            0/0            0/0            0/0   

    Otherinfo11014  
312            0/0  

[1 rows x 11025 columns]


In [39]:
coding_nonsynonymous.to_csv(f'{WORK_DIR}/coding_nonsynonymous.txt', sep = '\t', index = False)
coding_nonsynonymous.head()

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo11005,Otherinfo11006,Otherinfo11007,Otherinfo11008,Otherinfo11009,Otherinfo11010,Otherinfo11011,Otherinfo11012,Otherinfo11013,Otherinfo11014
312,9,129147457,129147457,C,T,exonic,PTPA,.,nonsynonymous SNV,"PTPA:NM_001271832:exon9:c.C878T:p.S293L,PTPA:N...",...,1/1,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0,0/0


## Calculate freq of coding and non-syn vars in cases versus controls

In [40]:
reduced_coding_nonsynonymous = coding_nonsynonymous[["Chr", "Start", "End", "Gene.refGene"]]
reduced_coding_nonsynonymous.to_csv(f'{WORK_DIR}/reduced_coding_nonsynonymous.txt', sep = '\t', index = False, header= False)

In [41]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

head reduced_coding_nonsynonymous.txt

9	129147457	129147457	PTPA


In [42]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_EUR --extract range reduced_coding_nonsynonymous.txt --assoc --out coding_nonsynonymous_pheno_PP2A --allow-no-sex --ci 0.95

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A.log.
Options in effect:
  --allow-no-sex
  --assoc
  --bfile pheno_PP2A_EUR
  --ci 0.95
  --extract range reduced_coding_nonsynonymous.txt
  --out coding_nonsynonymous_pheno_PP2A

14998 MB RAM detected; reserving 7499 MB for main workspace.
328 variants loaded from .bim file.
11002 people (6731 males, 4271 females) loaded from .fam.
10727 phenotype values loaded from .fam.
--extract range: 327 variants excluded.
--extract range: 1 variant remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 11002 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%

In [43]:
PP2A_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A.assoc', delim_whitespace=True)
PP2A_freq.head(10)

,CHR,SNP,BP,A1,F_A,F_U,A2,CHISQ,P,OR,SE,L95,U95
0,9,chr9:129147457:C:T,129147457,T,0.07145,0.07838,C,3.408,0.06489,0.9048,0.05422,0.8136,1.006


In [44]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_EUR --extract range reduced_coding_nonsynonymous.txt --make-bed --max-maf 0.05 --out coding_nonsynonymous_rare_0.05 --allow-no-sex

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_rare_0.05.log.
Options in effect:
  --allow-no-sex
  --bfile pheno_PP2A_EUR
  --extract range reduced_coding_nonsynonymous.txt
  --make-bed
  --max-maf 0.05
  --out coding_nonsynonymous_rare_0.05

14998 MB RAM detected; reserving 7499 MB for main workspace.
328 variants loaded from .bim file.
11002 people (6731 males, 4271 females) loaded from .fam.
10727 phenotype values loaded from .fam.
--extract range: 327 variants excluded.
--extract range: 1 variant remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 11002 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%

Error: All variants removed due to minor allele threshold(s)
(--maf/--max-maf/--mac/--max-mac).


CalledProcessError: Command 'b"\nWORK_DIR='/home/jupyter/PP2A_GP2/'\ncd $WORK_DIR\n\n/home/jupyter/tools/plink --bfile pheno_PP2A_EUR --extract range reduced_coding_nonsynonymous.txt --make-bed --max-maf 0.05 --out coding_nonsynonymous_rare_0.05 --allow-no-sex\n"' returned non-zero exit status 13.

In [45]:
PP2A_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A.assoc.adjusted', delim_whitespace=True)
PP2A_freq.head(10)

,CHR,SNP,UNADJ,GC,BONF,HOLM,SIDAK_SS,SIDAK_SD,FDR_BH,FDR_BY
0,9,chr9:129147457:C:T,0.06489,0.4995,0.06489,0.06489,0.06489,0.06489,0.06489,0.06489


In [46]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_EUR --extract range reduced_coding_nonsynonymous.txt --assoc --out coding_nonsynonymous_pheno_PP2A --allow-no-sex --adjust

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A.log.
Options in effect:
  --adjust
  --allow-no-sex
  --assoc
  --bfile pheno_PP2A_EUR
  --extract range reduced_coding_nonsynonymous.txt
  --out coding_nonsynonymous_pheno_PP2A

14998 MB RAM detected; reserving 7499 MB for main workspace.
328 variants loaded from .bim file.
11002 people (6731 males, 4271 females) loaded from .fam.
10727 phenotype values loaded from .fam.
--extract range: 327 variants excluded.
--extract range: 1 variant remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 11002 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%3

In [47]:
PP2A_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A.assoc.adjusted', delim_whitespace=True)
PP2A_freq.head(10)

,CHR,SNP,UNADJ,GC,BONF,HOLM,SIDAK_SS,SIDAK_SD,FDR_BH,FDR_BY
0,9,chr9:129147457:C:T,0.06489,0.4995,0.06489,0.06489,0.06489,0.06489,0.06489,0.06489


## Calculate freq of HMZ in cases versus controls

In [48]:
%%bash

WORK_DIR='/home/jupyter/PP2A_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_PP2A_EUR --extract range reduced_coding_nonsynonymous.txt --recode A --out coding_nonsynonymous_pheno_PP2A

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_PP2A.log.
Options in effect:
  --bfile pheno_PP2A_EUR
  --extract range reduced_coding_nonsynonymous.txt
  --out coding_nonsynonymous_pheno_PP2A
  --recode A

14998 MB RAM detected; reserving 7499 MB for main workspace.
328 variants loaded from .bim file.
11002 people (6731 males, 4271 females) loaded from .fam.
10727 phenotype values loaded from .fam.
--extract range: 327 variants excluded.
--extract range: 1 variant remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 11002 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38

In [49]:
PP2A_recode = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_PP2A.raw', delim_whitespace=True)
PP2A_recode.head()

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129147457:C:T_T
0,0,APGS_000001_s1,0,0,1,2,0.0
1,0,APGS_000002_s1,0,0,1,2,0.0
2,0,APGS_000003_s1,0,0,1,2,0.0
3,0,APGS_000005_s1,0,0,1,2,0.0
4,0,APGS_000006_s1,0,0,1,2,0.0


In [50]:
# Explore HMZs for chr9:129147457:C:T_T in cases
PP2A_hom_casos = PP2A_recode[(PP2A_recode['chr9:129147457:C:T_T'] == 2) & (PP2A_recode['PHENOTYPE'] == 2)]
PP2A_hom_casos.head(36)

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129147457:C:T_T
500,0,APGS_000651_s1,0,0,1,2,2.0
596,0,APGS_000771_s1,0,0,1,2,2.0
1157,0,APGS_001455_s1,0,0,2,2,2.0
1322,0,BCM_000181_s1,0,0,1,2,2.0
1327,0,BCM_000189_s1,0,0,2,2,2.0
2269,0,CORIELL_000366_s1,0,0,2,2,2.0
2445,0,CORIELL_000565_s1,0,0,1,2,2.0
2779,0,CORIELL_000971_s1,0,0,2,2,2.0
2793,0,CORIELL_000992_s1,0,0,1,2,2.0
2949,0,CORIELL_001185_s1,0,0,2,2,2.0


In [51]:
PP2A_hom_casos.shape

(36, 7)

In [56]:
# Explore HMZs for chr9:129147457:C:T_T in controls
PP2A_hom_controles = PP2A_recode[(PP2A_recode['chr9:129147457:C:T_T'] == 2) & (PP2A_recode['PHENOTYPE'] == 1)]
PP2A_hom_controles.head(36)

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129147457:C:T_T
1661,0,BCM_000623_s1,0,0,2,1,2.0
5545,0,CORIELL_004565_s1,0,0,2,1,2.0
5571,0,CORIELL_004595_s1,0,0,2,1,2.0
5665,0,CORIELL_004703_s1,0,0,2,1,2.0
5693,0,CORIELL_004748_s1,0,0,2,1,2.0
5698,0,CORIELL_004754_s1,0,0,1,1,2.0
5954,0,CORIELL_005055_s1,0,0,1,1,2.0
6408,0,CORIELL_005582_s1,0,0,2,1,2.0
6436,0,CORIELL_005612_s1,0,0,2,1,2.0
6668,0,CORIELL_005880_s1,0,0,2,1,2.0


In [53]:
PP2A_hom_controles.shape

(26, 7)

In [81]:
# Explore HETs for chr9:129147457:C:T_T in cases
PP2A_het_casos_129147457_EUR = PP2A_recode[(PP2A_recode['chr9:129147457:C:T_T'] == 1) & (PP2A_recode['PHENOTYPE'] == 2)]
PP2A_het_casos_129147457_EUR

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129147457:C:T_T
5,0,APGS_000007_s1,0,0,1,2,1.0
13,0,APGS_000018_s1,0,0,1,2,1.0
17,0,APGS_000024_s1,0,0,2,2,1.0
20,0,APGS_000027_s1,0,0,1,2,1.0
22,0,APGS_000029_s1,0,0,1,2,1.0
...,...,...,...,...,...,...,...
10981,0,UMD_000456_s1,0,0,1,2,1.0
10984,0,UMD_000460_s1,0,0,1,2,1.0
10985,0,UMD_000461_s1,0,0,1,2,1.0
10987,0,UMD_000464_s1,0,0,1,2,1.0


In [82]:
# Explore HETs for chr9:129147457:C:T_T in controls
PP2A_het_controles_129147457_EUR = PP2A_recode[(PP2A_recode['chr9:129147457:C:T_T'] == 1) & (PP2A_recode['PHENOTYPE'] == 1)]
PP2A_het_controles_129147457_EUR

,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129147457:C:T_T
1204,0,BCM_000035_s1,0,0,2,1,1.0
1261,0,BCM_000099_s1,0,0,2,1,1.0
1302,0,BCM_000149_s1,0,0,2,1,1.0
1326,0,BCM_000186_s1,0,0,1,1,1.0
1360,0,BCM_000237_s1,0,0,2,1,1.0
...,...,...,...,...,...,...,...
10827,0,UMD_000249_s1,0,0,2,1,1.0
10905,0,UMD_000352_s1,0,0,2,1,1.0
10911,0,UMD_000359_s1,0,0,2,1,1.0
10929,0,UMD_000384_s1,0,0,2,1,1.0


In [66]:
!pip install rpy2
%load_ext rpy2.ipython

In [73]:
import rpy2
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [83]:
PP2A_het_casos_129147457_EUR.to_csv('PP2A_het_casos_129147457_EUR.csv')

In [84]:
temp = pd.read_csv('PP2A_het_casos_129147457_EUR.csv')
temp

,Unnamed: 0,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129147457:C:T_T
0,5,0,APGS_000007_s1,0,0,1,2,1.0
1,13,0,APGS_000018_s1,0,0,1,2,1.0
2,17,0,APGS_000024_s1,0,0,2,2,1.0
3,20,0,APGS_000027_s1,0,0,1,2,1.0
4,22,0,APGS_000029_s1,0,0,1,2,1.0
...,...,...,...,...,...,...,...,...
885,10981,0,UMD_000456_s1,0,0,1,2,1.0
886,10984,0,UMD_000460_s1,0,0,1,2,1.0
887,10985,0,UMD_000461_s1,0,0,1,2,1.0
888,10987,0,UMD_000464_s1,0,0,1,2,1.0


In [85]:
PP2A_het_controles_129147457_EUR.to_csv('PP2A_het_controles_129147457_EUR.csv')

In [86]:
temp = pd.read_csv('PP2A_het_controles_129147457_EUR.csv')
temp

,Unnamed: 0,FID,IID,PAT,MAT,SEX,PHENOTYPE,chr9:129147457:C:T_T
0,1204,0,BCM_000035_s1,0,0,2,1,1.0
1,1261,0,BCM_000099_s1,0,0,2,1,1.0
2,1302,0,BCM_000149_s1,0,0,2,1,1.0
3,1326,0,BCM_000186_s1,0,0,1,1,1.0
4,1360,0,BCM_000237_s1,0,0,2,1,1.0
...,...,...,...,...,...,...,...,...
539,10827,0,UMD_000249_s1,0,0,2,1,1.0
540,10905,0,UMD_000352_s1,0,0,2,1,1.0
541,10911,0,UMD_000359_s1,0,0,2,1,1.0
542,10929,0,UMD_000384_s1,0,0,2,1,1.0


In [76]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
covs <- fread("master_key_release3_final.csv", header =T)
covs

                  GP2ID         GP2sampleID manifest_id phenotype pheno_for_qc
    1: SYNAPS-KZ_000001 SYNAPS-KZ_000001_s1          m1   Control            1
    2: SYNAPS-KZ_000002 SYNAPS-KZ_000002_s1          m1   Control            1
    3: SYNAPS-KZ_000003 SYNAPS-KZ_000003_s1          m1   Control            1
    4: SYNAPS-KZ_000004 SYNAPS-KZ_000004_s1          m1   Control            1
    5: SYNAPS-KZ_000005 SYNAPS-KZ_000005_s1          m1   Control            1
   ---                                                                        
16853:      PPMI_000789      PPMI_000789_s1          m2     Other           -9
16854:      PPMI_000781      PPMI_000781_s1          m2     Other           -9
16855:      PPMI_000818      PPMI_000818_s1          m2     Other           -9
16856:      PPMI_000841      PPMI_000841_s1          m2     Other           -9
16857:      PPMI_000808      PPMI_000808_s1          m2     Other           -9
                                  other_pheno sex_fo

In [77]:
%%R
colnames(covs)[2]  <- "IID"  
covs

                  GP2ID                 IID manifest_id phenotype pheno_for_qc
    1: SYNAPS-KZ_000001 SYNAPS-KZ_000001_s1          m1   Control            1
    2: SYNAPS-KZ_000002 SYNAPS-KZ_000002_s1          m1   Control            1
    3: SYNAPS-KZ_000003 SYNAPS-KZ_000003_s1          m1   Control            1
    4: SYNAPS-KZ_000004 SYNAPS-KZ_000004_s1          m1   Control            1
    5: SYNAPS-KZ_000005 SYNAPS-KZ_000005_s1          m1   Control            1
   ---                                                                        
16853:      PPMI_000789      PPMI_000789_s1          m2     Other           -9
16854:      PPMI_000781      PPMI_000781_s1          m2     Other           -9
16855:      PPMI_000818      PPMI_000818_s1          m2     Other           -9
16856:      PPMI_000841      PPMI_000841_s1          m2     Other           -9
16857:      PPMI_000808      PPMI_000808_s1          m2     Other           -9
                                  other_pheno sex_fo

In [90]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
PP2A_het_casos_129147457_EUR <- fread("PP2A_het_casos_129147457_EUR.csv", header =T)
PP2A_het_casos_129147457_EUR
temp <- merge(covs, PP2A_het_casos_129147457_EUR, by="IID")
head(temp)
write.table(temp, file = "PP2A_het_casos_129147457_EUR_extracted.txt", quote = F, row.names = F, sep = "\t")
PP2A_het_casos_129147457_EUR_extracted <- fread("PP2A_het_casos_129147457_EUR_extracted.txt", header =T)
PP2A_het_casos_129147457_EUR_extracted

                IID       GP2ID manifest_id phenotype pheno_for_qc other_pheno
  1: APGS_000007_s1 APGS_000007          m1        PD            2          PD
  2: APGS_000018_s1 APGS_000018          m1        PD            2          PD
  3: APGS_000024_s1 APGS_000024          m1        PD            2          PD
  4: APGS_000027_s1 APGS_000027          m1        PD            2          PD
  5: APGS_000029_s1 APGS_000029          m1        PD            2          PD
 ---                                                                          
886:  UMD_000456_s1  UMD_000456          m1        PD            2          PD
887:  UMD_000460_s1  UMD_000460          m1        PD            2          PD
888:  UMD_000461_s1  UMD_000461          m1        PD            2          PD
889:  UMD_000464_s1  UMD_000464          m1        PD            2          PD
890:  UMD_000466_s1  UMD_000466          m1        PD            2          PD
     sex_for_qc age age_of_onset age_at_diagnosis ag

In [91]:
%%R
setwd('/home/jupyter/PP2A_GP2/')
library(data.table)
PP2A_het_controles_129147457_EUR <- fread("PP2A_het_controles_129147457_EUR.csv", header =T)
PP2A_het_controles_129147457_EUR
temp <- merge(covs, PP2A_het_controles_129147457_EUR, by="IID")
head(temp)
write.table(temp, file = "PP2A_het_controles_129147457_EUR_extracted.txt", quote = F, row.names = F, sep = "\t")
PP2A_het_controles_129147457_EUR_extracted <- fread("PP2A_het_controles_129147457_EUR_extracted.txt", header =T)
PP2A_het_controles_129147457_EUR_extracted

               IID      GP2ID manifest_id phenotype pheno_for_qc other_pheno
  1: BCM_000035_s1 BCM_000035          m1   Control            1     Control
  2: BCM_000099_s1 BCM_000099          m1   Control            1     Control
  3: BCM_000149_s1 BCM_000149          m1   Control            1     Control
  4: BCM_000186_s1 BCM_000186          m1   Control            1     Control
  5: BCM_000237_s1 BCM_000237          m1   Control            1     Control
 ---                                                                        
540: UMD_000249_s1 UMD_000249          m1   Control            1     Control
541: UMD_000352_s1 UMD_000352          m1   Control            1     Control
542: UMD_000359_s1 UMD_000359          m1   Control            1     Control
543: UMD_000384_s1 UMD_000384          m1   Control            1     Control
544: UMD_000454_s1 UMD_000454          m1   Control            1     Control
     sex_for_qc age age_of_onset age_at_diagnosis age_at_death  race_for_qc


## Save out results..!

In [89]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR} {WORKSPACE_BUCKET}')

Executing: gsutil -mu terra-02e7cd23 cp -r /home/jupyter/PP2A_GP2/ gs://fc-a2382a6f-59cf-4cf3-b70e-8cd83806d5e0


Copying file:///home/jupyter/PP2A_GP2/reduced_coding_nonsynonymous.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/PP2A_GP2/PP2A_het_controles_129147457_EUR_extracted.txt [Content-Type=text/plain]...
Copying file:///home/jupyter/PP2A_GP2/coding_nonsynonymous_AJ_rare_0.05.log [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A_CAS.log [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A_AFR.fam [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/coding_nonsynonymous_AAC_rare_0.05.bed [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A_EAS.log [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A_AJ.annovar.avinput [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A.vcf [Content-Type=text/vcard]...
Copying file:///home/jupyter/PP2A_GP2/pheno_PP2A.annov